Primero vamos a importar todas las librerias a usar

In [88]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font
from datetime import datetime
import math
from openpyxl.styles import Side, Border
import numpy as np

Segundo, vamos a importar la base de datos a usar

In [89]:
# Cargar archivo usando la hoja "Data"
df = pd.read_excel(
    "/Users/rodrigoluisyauliblas/Downloads/Spot 2 May.xlsx", #Cambiar la ubicación de origen del archivo que contenga la base de datos en Excel "/Users/rodrigoluisyauliblas/Downloads/Spot 2 May.xlsx"
    sheet_name="Data" #Si en caso la hoja que contenga los inputs tenga otro nombre, cambiar el nombre "Data" por la hoja que ocntenga los datos
)

# Mostrar información del DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Hora    265 non-null    object 
 1   Precio  265 non-null    float64
 2   Monto   265 non-null    int64  
 3   R       265 non-null    object 
dtypes: float64(1), int64(1), object(2)
memory usage: 8.4+ KB


Tercero, En la columna Hora no está especificado como Dtype de tiempo, por lo que se va a convertir a datatime para luego filtrar por lo solicitado, esto se hace ya que al intentar filtrar sin un paso previo de datetime este no se puede filtrar, ya con este paso el filtro funciona.

In [90]:
# convertir a datetime y luego extraer solo la hora
df['Hora'] = pd.to_datetime(df['Hora'], format='%H:%M:%S').dt.time

#Filtrar por tiempo solicitado
en_hora = df[(df['Hora'] >= time(9, 0)) & (df['Hora'] <= time(13, 30))] #En este apartado se puede cambiar el filtro de tiempo

3.1 para corroborar que el filtro ha funcionado se va a comprobar el minimo y maximo de la base completa y del filtro usado

In [91]:
# Hallar min/max de ambas base de datos
resumen = pd.DataFrame({
    "Variable": ["Hora", "Precio", "Monto"],
    
    "Min (df)": [
        df["Hora"].min(),
        df["Precio"].min(),
        df["Monto"].min()
    ],
    "Max (df)": [
        df["Hora"].max(),
        df["Precio"].max(),
        df["Monto"].max()
    ],
    
    "Min (en_hora)": [
        en_hora["Hora"].min(),
        en_hora["Precio"].min(),
        en_hora["Monto"].min()
    ],
    "Max (en_hora)": [
        en_hora["Hora"].max(),
        en_hora["Precio"].max(),
        en_hora["Monto"].max()
    ]
})

print("\n=== MIN Y MAX DE df vs en_hora ===\n")
print(resumen.to_string(index=False))


=== MIN Y MAX DE df vs en_hora ===

Variable Min (df) Max (df) Min (en_hora) Max (en_hora)
    Hora 08:32:09 13:43:08      09:44:09      13:29:33
  Precio    2.633     2.64         2.633          2.64
   Monto   500000 20000000        500000      20000000


Cuarto, vamos a ordenar la base de datos desde la operación más temprana hasta la operación más cercana al corte de tiempo

In [92]:
#Ordenamiento de la base de datos filtrada
en_hora = en_hora.sort_values(by="Hora").reset_index(drop=True)

#Ver los primeros 10 registros de la base de datos
en_hora.head(10) #En caso solo se desee ver la primera fila de datos cambiar el 10 por el 1

,Hora,Precio,Monto,R
0,09:44:09,2.637,1000000,P
1,09:44:46,2.637,1000000,P
2,09:45:59,2.637,1000000,P
3,09:46:04,2.637,1000000,P
4,09:46:05,2.637,1000000,P
5,10:06:32,2.636,1000000,P
6,10:10:10,2.637,500000,R
7,10:13:57,2.636,1000000,P
8,10:13:58,2.636,5000000,P
9,10:13:58,2.636,1000000,P


Quinto, configuración del reporte de Excel a crear

In [93]:
# Configuración
MAX_ROWS = 65       # total por bloque
HEADER_ROWS = 2     # título + encabezado
DATA_PER_BLOCK = MAX_ROWS - HEADER_ROWS  # 63 registros por bloque

# Creación del arhcivo Excel
sheet_date = datetime.now().strftime("%Y-%m-%d")
wb = Workbook()
ws = wb.active

# Nombre de la hoja de Excel
ws.title = "Reporte" # En caso se desee colocar otro nombre a la hoja de Excel habría que cambiar el nombre "Reporte" por el nombre que se desee

Sexto, configuración de la hoja "Reporte" para el ordenamiento de datos así como el cuadro resumen

In [94]:
# Título
ws["A1"] = f"REPORTE DE OPERACIONES – {sheet_date}" #{sheet_date} es la función con la fecha actual, con este se puede tener el título con la fecha del día en que se genera el reporte
ws["A1"].font = Font(bold=True, size=14)

# Calculo de bloques
n = len(en_hora)
blocks = math.ceil(n / DATA_PER_BLOCK)
columns_per_block = 4

# Encabezados
headers = ["Hora", "Precio", "Monto", "R"] # Si en caso se va a tener más encabezados (más columnas) colocar el nombre de las columnas, no obstante, según el tipo de datos el código se podrá ver afectado, para mayor consultas contactarse mediante mail con el responsable del código.

def write_block_headers(start_col):
    for i, h in enumerate(headers):
        cell = ws.cell(row=2, column=start_col + i)
        cell.value = h
        cell.font = Font(bold=True)

# Escritura de bloques
for b in range(blocks):

    start_row = 3
    start_col = 1 + b * columns_per_block

    # encabezados
    write_block_headers(start_col)

    # rango del bloque
    start_idx = b * DATA_PER_BLOCK
    end_idx = min(start_idx + DATA_PER_BLOCK, n)

    block_df = en_hora.iloc[start_idx:end_idx]

    # escribir registros
    r = start_row
    for _, row in block_df.iterrows():

        # Hora
        ws.cell(row=r, column=start_col).value = row["Hora"]

        # Precio (4 decimales)
        precio_cell = ws.cell(row=r, column=start_col + 1)
        precio_cell.value = float(row["Precio"])
        precio_cell.number_format = "0.0000" # Si deseas mostrar solo 2 o 3 decimales, debes ajustar el formato numérico agregando la cantidad de ceros que corresponde a los decimales que quieres visualizar.

        # Monto (miles)
        monto_cell = ws.cell(row=r, column=start_col + 2)
        monto_cell.value = int(row["Monto"])
        monto_cell.number_format = "#,##0"

        # R
        ws.cell(row=r, column=start_col + 3).value = row["R"]

        r += 1

6.1 Calculo de la volatilidad

In [95]:
# volatilidad simple = desviación estándar de precios
volatilidad = en_hora["Precio"].std(ddof=0)

print("Volatilidad (desviación estándar de precios):", volatilidad)
print("Volatilidad redondeada a 4 decimales:", round(volatilidad, 4))

Volatilidad (desviación estándar de precios): 0.0014866692288179034
Volatilidad redondeada a 4 decimales: 0.0015


6.2 Resumen 

In [96]:
# Resumen
summary_rows = [
    ("TC Min", en_hora["Precio"].min(), "0.0000"), # Si deseas mostrar solo 2 o 3 decimales, debes ajustar el formato numérico agregando la cantidad de ceros que corresponde a los decimales que quieres visualizar.
    ("TC Max", en_hora["Precio"].max(), "0.0000"), # Si deseas mostrar solo 2 o 3 decimales, debes ajustar el formato numérico agregando la cantidad de ceros que corresponde a los decimales que quieres visualizar.
    ("Promedio", en_hora["Precio"].mean(), "0.0000"), # Si deseas mostrar solo 2 o 3 decimales, debes ajustar el formato numérico agregando la cantidad de ceros que corresponde a los decimales que quieres visualizar.
    ("Monto Total", en_hora["Monto"].sum(), "#,##0"),
    ("Volatilidad", volatilidad, "0.0000") # Si deseas mostrar solo 2 o 3 decimales, debes ajustar el formato numérico agregando la cantidad de ceros que corresponde a los decimales que quieres visualizar.
]

# ubicación
last_block = blocks - 1
last_block_start_col = 1 + last_block * columns_per_block
last_data_rows = n - (last_block * DATA_PER_BLOCK)
last_data_row_excel = 2 + last_data_rows

# si se pasa de fila 65, siguiente bloque
if last_data_row_excel + len(summary_rows) > MAX_ROWS:
    last_block += 1
    last_block_start_col = 1 + last_block * columns_per_block
    summary_row = 3
else:
    summary_row = last_data_row_excel + 1

start_r = summary_row

# escribir resumen
for title, value, fmt in summary_rows:
    ws.cell(row=start_r, column=last_block_start_col).value = title
    vcell = ws.cell(row=start_r, column=last_block_start_col + 1)
    vcell.value = value
    vcell.number_format = fmt
    start_r += 1

# delimitación del cuadro
end_r = start_r - 1
start_c = last_block_start_col
end_c = last_block_start_col + 1

thin = Side(border_style="thin", color="000000")
border_top = Border(top=thin)
border_bottom = Border(bottom=thin)
border_left = Border(left=thin)
border_right = Border(right=thin)

# bordes horizontales (solo externos)
for col in range(start_c, end_c + 1):
    ws.cell(row=summary_row, column=col).border = ws.cell(row=summary_row, column=col).border + border_top
    ws.cell(row=end_r, column=col).border = ws.cell(row=end_r, column=col).border + border_bottom

# bordes verticales (solo externos)
for row in range(summary_row, end_r + 1):
    ws.cell(row=row, column=start_c).border = ws.cell(row=row, column=start_c).border + border_left
    ws.cell(row=row, column=end_c).border = ws.cell(row=row, column=end_c).border + border_right

Séptimo, guardar y exportar el "Reporte"

In [97]:
# Guardar
wb.save("Rodrigo Yauli.xlsx")
print("Rodrigo Yauli.xlsx se generó correctamente.")

Rodrigo Yauli.xlsx se generó correctamente.
